In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import torch.nn.functional as F
import unicodedata
import io
import json
import re
from google.colab import files


In [2]:
# Download the training set.
!wget https://raw.githubusercontent.com/nitinpunjabi/nlp-demystified/main/datasets/hun_eng_pairs/hun_eng_pairs_train.txt

--2023-05-02 17:45:01--  https://raw.githubusercontent.com/nitinpunjabi/nlp-demystified/main/datasets/hun_eng_pairs/hun_eng_pairs_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5518306 (5.3M) [text/plain]
Saving to: ‘hun_eng_pairs_train.txt’

hun_eng_pairs_train 100%[===================>]   5.26M  --.-KB/s    in 0.1s    

2023-05-02 17:45:02 (45.0 MB/s) - ‘hun_eng_pairs_train.txt’ saved [5518306/5518306]



In [3]:
with open('hun_eng_pairs_train.txt') as file:
  train = [line.rstrip() for line in file]

In [4]:
train[:3],len(train)

(["Teszek rá, mit mondasz!<sep>I don't care what you say.",
  'Több olyan ember kell nekünk a csapatba, mint amilyen te vagy.<sep>We need more people like you on our team.',
  'Vigyázz a gyerekeimre!<sep>Take care of my children.'],
 88647)

In [5]:
# Separate the input (Hungarian) and target (English) sentences into separate lists.
SEPARATOR = '<sep>'
train_input, train_target = map(list, zip(*[pair.split(SEPARATOR) for pair in train]))

In [6]:
print(train_input[:3])
print(train_target[:3])

['Teszek rá, mit mondasz!', 'Több olyan ember kell nekünk a csapatba, mint amilyen te vagy.', 'Vigyázz a gyerekeimre!']
["I don't care what you say.", 'We need more people like you on our team.', 'Take care of my children.']


In [7]:
print("\u00E1", "\u0061\u0301")

á á


In [8]:
import unicodedata
import re
# Unicode normalization
def normalize_unicode(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

In [9]:
def preprocess_sentence(s):
  s = normalize_unicode(s)
  s = re.sub(r"([?.!,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = s.strip()
  return s

In [10]:
# Preprocess both the source and target sentences.
train_preprocessed_input = [preprocess_sentence(s) for s in train_input]
train_preprocessed_target = [preprocess_sentence(s) for s in train_target]

In [11]:
train_preprocessed_input[:3],train_preprocessed_target[:3],

(['Teszek ra , mit mondasz !',
  'Tobb olyan ember kell nekunk a csapatba , mint amilyen te vagy .',
  'Vigyazz a gyerekeimre !'],
 ["I don't care what you say .",
  'We need more people like you on our team .',
  'Take care of my children .'])

In [12]:
def tag_target_sentences(sentences):
  tagged_sentences = map(lambda s: (' ').join(['<sos>', s, '<eos>']), sentences)
  return list(tagged_sentences)

In [13]:
train_tagged_preprocessed_target = tag_target_sentences(train_preprocessed_target)
train_tagged_preprocessed_target[0]

"<sos> I don't care what you say . <eos>"

In [14]:
np.array(train_preprocessed_input).shape

(88647,)

In [15]:
def tokenise(line):
	ans=[]
	punc = [',', '.', '"', "'", '/', '*', ',', '?', '!', '-', '\n', '“', '”', '_', '&', '\ufeff', '&', ';', ":",'#','$','%','&','(',')','*','+','-','/',':',';','=','@',',','[\\]','^','_','`{|}~','\t']
	for el in line:
		if el in punc:
			line = line.replace(el, "")
	
	line=line.lower()
	return line.split()
	raise NotImplementedError

In [16]:
inp=np.array
a=tokenise(train_preprocessed_target[1])
a=np.array([a])
a.reshape(9,1)
a[0]

array(['we', 'need', 'more', 'people', 'like', 'you', 'on', 'our', 'team'],
      dtype='<U6')

In [17]:
np.array([1,2,3]).shape

(3,)

In [18]:
sent=np.array([tokenise(train_preprocessed_input[0])])
sent=np.append(sent,'<pad>')
len(sent[0])

6

In [19]:
input=np.array([tokenise(train_preprocessed_input[0])])
pad=np.full((50-len(input[0])),'<pad>')
sent=np.concatenate((input.squeeze(0),pad),axis=0)

In [20]:
train_input=train_preprocessed_input[:1000]
train_target=train_tagged_preprocessed_target[:1000]
test_input=train_preprocessed_input[1000:1200]
test_target=train_tagged_preprocessed_target[1000:1200]

In [21]:
de_vocab=set()
for line in train_input:
  sent=np.array([tokenise(line)])
  for x in sent[0]:
    de_vocab.add(x)

In [22]:
en_vocab=set()
for line in train_target:
  sent=np.array([tokenise(line)])
  for x in sent[0]:
    en_vocab.add(x)

In [23]:
!pip install ordered-set
from ordered_set import OrderedSet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
vg=list(de_vocab)
vg.insert(0,'<pad>')
de_vocab=OrderedSet(vg)

ve=list(en_vocab)
ve.insert(0,'<pad>')
en_vocab=OrderedSet(ve)

In [25]:
de_word2ix = {word: i for i, word in enumerate(de_vocab)}
de_ix2word = {i: word for i, word in enumerate(de_vocab)}
en_word2ix = {word: i for i, word in enumerate(en_vocab)}
en_ix2word = {i: word for i, word in enumerate(en_vocab)}

In [26]:
def tensor(list):
  return torch.from_numpy(np.array(list))

In [27]:
def de_generate_index(line):
  new_line=[]
  sent=tokenise(line)
  for x in sent:
   new_line.append(de_word2ix[x])
  return tensor(new_line)

In [28]:
def en_generate_index(line):
  new_line=[]
  sent=tokenise(line)
  for x in sent:
   new_line.append(en_word2ix[x])
  return tensor(new_line)

In [29]:
len(train_input),len(train_target)

(1000, 1000)

In [30]:
input=train_input
target=train_target

In [31]:
de_generate_index(input[0]),en_generate_index(target[0])

(tensor([ 585,   26, 1616, 1550]),
 tensor([1032,  964,  962,  955,  169, 1267,  857,  367]))

In [32]:
def longest_seq(corpus):
  c=len(tokenise(corpus[0]))
  for sent in corpus:
    line=tokenise(sent)
    if c<len(line):c=len(line)
  return c
lg=max(longest_seq(input),longest_seq(target))
lg

22

In [33]:
def pad(line,longest_seq_length):
  pad=torch.zeros((longest_seq_length-len(line)))
  return torch.cat((line,pad))

In [34]:
a=list(tokenise(target[0]))
a.append('p')
a

['<sos>', 'i', 'dont', 'care', 'what', 'you', 'say', '<eos>', 'p']

In [35]:
def pad_sent(line):
  a=list(tokenise(line))
  for i in range(lg-len(a)):
    a.append('<pad>')
  return a

pad_sent(target[0])

['<sos>',
 'i',
 'dont',
 'care',
 'what',
 'you',
 'say',
 '<eos>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

In [36]:
def de_index(batch,batch_size):
  if(batch_size==1):return pad(de_generate_index(batch),lg).unsqueeze(1).type(torch.LongTensor)
  bs=pad(de_generate_index(batch[0]),lg)
  for i in range(1,batch_size):
    bs=torch.vstack((bs,pad(de_generate_index(batch[i]),lg)))
  bs=bs.permute(1,0)
  return bs.type(torch.LongTensor)

def en_index(batch,batch_size):
  if(batch_size==1):return pad(en_generate_index(batch),lg).unsqueeze(1).type(torch.LongTensor)
  bs=pad(en_generate_index(batch[0]),lg)
  for i in range(1,batch_size):
    bs=torch.vstack((bs,pad(en_generate_index(batch[i]),lg)))
  bs=bs.permute(1,0)
  return bs.type(torch.LongTensor)

In [37]:
de_index(input[:5],5).shape,de_index(input[0],1).shape

(torch.Size([22, 5]), torch.Size([22, 1]))

In [38]:
def initialize_embeddings(de_vocab_size,en_vocab_size ,embedding_dim):
    """
    Initialize source and target embeddings
    
    Args:
    vocab_size: size of the vocabulary
    embedding_dim: size of the embedding dimension
    
    Returns:
    source_embed: source embedding layer
    target_embed: target embedding layer
    """
    
    # initialize source embedding layer
    source_embed = nn.Embedding(de_vocab_size, embedding_dim)
    
    # initialize target embedding layer
    target_embed = nn.Embedding(en_vocab_size, embedding_dim)
    
    # return source and target embeddings
    return source_embed, target_embed

In [39]:
source_embed, target_embed = initialize_embeddings(de_vocab_size=len(de_vocab),en_vocab_size=len(en_vocab), embedding_dim=30)

In [40]:
target_embed(en_index(target[:5],5))

tensor([[[-0.4429, -0.4117, -0.8145,  ...,  1.0872,  0.5805,  1.8883],
         [-0.4429, -0.4117, -0.8145,  ...,  1.0872,  0.5805,  1.8883],
         [-0.4429, -0.4117, -0.8145,  ...,  1.0872,  0.5805,  1.8883],
         [-0.4429, -0.4117, -0.8145,  ...,  1.0872,  0.5805,  1.8883],
         [-0.4429, -0.4117, -0.8145,  ...,  1.0872,  0.5805,  1.8883]],

        [[ 0.8336, -1.6348, -0.6143,  ..., -1.0834,  0.0617, -1.6126],
         [-1.4464,  1.8942, -0.3635,  ...,  1.3698,  0.0623, -0.2304],
         [ 1.1791, -0.3261, -0.1692,  ...,  0.9743,  0.2426,  0.6261],
         [ 0.7874,  0.4134, -1.3717,  ...,  0.9291,  0.4896, -0.4666],
         [-0.3999,  0.5301, -0.4987,  ...,  1.2097, -1.5891,  0.2441]],

        [[-1.3249, -0.8279, -0.1519,  ..., -0.3500,  0.6499,  1.2928],
         [-0.4177,  0.7122,  1.4552,  ...,  0.4658, -0.2616,  0.8183],
         [ 0.7263,  1.2240,  1.1188,  ...,  1.3743,  1.1240, -0.5373],
         [-0.6855, -0.3277,  0.9495,  ...,  0.5975, -1.3113,  0.3678],
  

In [41]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size,num_layers,encoder_dropout):
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers=num_layers
        self.lstm = nn.LSTM(input_size, hidden_size,num_layers, bidirectional=True)
        self.dropout=nn.Dropout(encoder_dropout)
        self.lin_hid=nn.Linear(hidden_size*2,hidden_size)
        self.lin_cell=nn.Linear(hidden_size*2,hidden_size)
    def forward(self, input_seq,batch_size):
        #embedded=seq_len,batchsize,embeddingsize
        embedded = self.dropout(source_embed(de_index(input_seq,batch_size)))

        output, (hidden, cell) = self.lstm(embedded)
        hidden = self.lin_hid(torch.cat((hidden[0:1],hidden[1:2]), dim=2))
        cell = self.lin_cell(torch.cat((cell[0:1],cell[1:2]), dim=2))
        return output, hidden, cell
        #output=seq_len,batchsize,2*hiddensize
        #hidden=2*numlayers,batchsizze,hiddensize=128
        #cell=2*numlayers,batchsizze,hiddensize=128

In [42]:
# Create an instance of the Encoder model with the pre-trained embedding matrix
encoder = Encoder(input_size=30, hidden_size=128,num_layers=1,encoder_dropout=0.5)
input_seq = input[:5] # example input sequence of shape (batch_size, sequence_length)
encoder_output, encoder_hidden, encoder_cell = encoder(input_seq,5)
encoder_output.shape,encoder_hidden.shape,encoder_cell.shape

(torch.Size([22, 5, 256]), torch.Size([1, 5, 128]), torch.Size([1, 5, 128]))

In [43]:
class Decoder(nn.Module):
    def __init__(self,input_size,hidden_size, output_size, num_layers,decoder_dropout):
        super(Decoder, self).__init__()

        self.num_layers=num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=False)
        self.tanh=nn.Tanh()
        self.output_layer = nn.Linear(hidden_size, output_size)
        self.dropout=nn.Dropout(decoder_dropout)
        self.weight = nn.Parameter(torch.Tensor(hidden_size*3, 1))
        self.fc=nn.Linear(3*hidden_size,hidden_size)
    def forward(self, input_seq,encoder_output, hidden,cell):
        #input_seq:shape=batchsize
        #input_seq.unsqeeze(0):shape=1,batchsize
        embedded = self.dropout(target_embed(input_seq.unsqueeze(0)))
        decoder_output, (decoder_hidden, decoder_cell) = self.lstm(embedded, (hidden, cell))
        new_dec_out=decoder_hidden.repeat(lg,1,1).permute(1,0,2)#5,22,128
        e=encoder_output.permute(1,0,2)#5,22,256
        de=torch.cat((new_dec_out,e),dim=2)#5,22,384
        energy=torch.matmul(de,self.weight)#5,22,1
        aw=torch.softmax(energy,dim=1)#5,22,1
        context=torch.sum(e*aw,dim=1)#5,256
        #dec_hid_context=self.tanh(torch.concat((context.unsqueeze(dim=0),decoder_hidden),dim=2))#1,5,384
        #new_output=self.fc(dec_hid_context)
        new_output=self.fc(self.tanh(torch.concat((context.unsqueeze(dim=0),decoder_hidden),dim=2)))
        # Pass the LSTM output through the output layer to get the output
        pred = self.output_layer(new_output)
        pred.squeeze(0)
        return pred,hidden,cell
        #prde=output=1,en_vocabsize
        #hidden=num_layers,hiddensize
        #cell=num_layers,hiddensize

In [44]:
decoder = Decoder(input_size=30, hidden_size=128, output_size=len(en_vocab), num_layers=1,decoder_dropout=0.5)
input_seq = en_index(target[:5],5)[0] # replace with your input
 # use the final hidden state from the encoder
decoder_output, decoder_hidden,decoder_cell = decoder(input_seq,encoder_output, encoder_hidden,encoder_cell)
decoder_output.shape,decoder_hidden.shape,decoder_cell.shape

(torch.Size([1, 5, 1291]), torch.Size([1, 5, 128]), torch.Size([1, 5, 128]))

In [45]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, target_seq,batch_size):
        target_len = lg
        target_vocab_size = len(en_vocab)
        outputs = torch.zeros(target_len, batch_size, target_vocab_size)
        encoder_output, encoder_hidden, encoder_cell = self.encoder(input_seq,batch_size)
        decoder_input = target_seq[0]
        for t in range(1, target_len):
            output, hidden, cell = self.decoder(decoder_input,encoder_output, encoder_hidden, encoder_cell)
            outputs[t] = output
            decoder_input = target_seq[t]
        return outputs#shape=seq_len,batchsize,en_vocabsize

In [46]:
encoder = Encoder(input_size=30, hidden_size=128,num_layers=1,encoder_dropout=0.5)
decoder = Decoder(input_size=30, hidden_size=128, output_size=len(en_vocab), num_layers=1,decoder_dropout=0.5)
model=Seq2Seq(encoder,decoder)

out=model(input[:5],en_index(target[:5],5),batch_size=5)
out.shape

torch.Size([22, 5, 1291])

In [47]:
torch.argmax(out[1][0])

tensor(332)

In [48]:
num_epochs= 3
learning_rate = 0.0000000001
batch_size = 5

In [49]:
encoder = Encoder(input_size=30, hidden_size=1024,num_layers=1,encoder_dropout=0.2)
decoder = Decoder(input_size=30, hidden_size=1024, output_size=len(en_vocab), num_layers=1,decoder_dropout=0.2)
model=Seq2Seq(encoder,decoder)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
loss_fn = nn.CrossEntropyLoss()

In [50]:
def create_target_tensor(target_sentence,batch_size):
    
    target_tensor = torch.zeros(batch_size,len(en_vocab))
    for n in range(batch_size):
      target=en_index(target_sentence[n],1)
      for i,word in enumerate(target):
        target_tensor[n][torch.IntTensor.item(word)]=1
    target_tensor=target_tensor.type(torch.long)
    return target_tensor

In [51]:
loss_fn(out.permute(1,0,2),create_target_tensor(target[:5],5))

tensor(3.0902, grad_fn=<NllLoss2DBackward0>)

In [52]:
# Driving training loop
BATCH=10
model.train()
#for epoch in tqdm(range(1,num_epochs+1)):
for epoch in range(3):
    total_epoch_loss=0
    # Iterate through train dataset
    for i in range(0,961,BATCH):
        # 1. forward pass the inputs through the model
        output =model(input[i:i+BATCH],en_index(target[i:i+BATCH],BATCH),batch_size=BATCH).permute(1,0,2)
        trg=create_target_tensor(target[i:i+BATCH],BATCH)

        optimizer.zero_grad()
        
        loss = loss_fn(output,trg)                                 
        total_epoch_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1)
        # 4. update the parameters
        optimizer.step()
        print(i)

    print(f"Epoch: [{epoch}/{num_epochs}] Epoch Loss: {total_epoch_loss}")

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810


KeyboardInterrupt: ignored

In [ ]:
def translate_sentence(model, sentence,max_length=lg):


    with torch.no_grad():
        encoder_output,encoder_hidden, encoder_cell = model.encoder(sentence,1)
    out=[89]
    for i in range(max_length):
        previous_word = torch.tensor(out[-1])

        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word.unsqueeze(0),encoder_output,encoder_hidden,encoder_cell)
            best_guess=torch.IntTensor.item(torch.argmax(output[0][0]))
        out.append(best_guess)

        #if torch.IntTensor.item(torch.argmax(output[0][0])) == en_word2ix['<eos>']:
         #   break

    translated_sen = [en_ix2word[idx] for idx in out]

    return translated_sen[1:]

In [ ]:
sent=input[999]
translate_sentence(model,sent)

In [ ]:
from torchtext.data.metrics import bleu_score

In [ ]:
def bleu(data, model, german, english, device):
    targets= []
    outputs= []

    for eg in data:
        src = vars(eg)['src']
        trg = var(eg)['trg']

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1] # eos removed

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

In [ ]:
'''
trg=trg.type(torch.LongTensor)
out=out.type(torch.LongTensor)
scores = torch.zeros((31,5,10612))  # shape: (batch_size=10, num_classes=5)
labels = torch.ones((31,5,10612),dtype=torch.long)  # shape: (batch_size=10)

# Define the cross-entropy loss function

# Compute the loss
loss_fn(scores, labels)
'''